# Производная, дифференцирование.

[Лекция "Предел и производная"](https://docs.google.com/presentation/d/e/2PACX-1vQrmFcOnz5M_88Hg4XD_hsP7AYgiwGcFl14JZfKo8Cqv8wts1Gj8_Ebd4fP7_zKhso32dE9HvSYQuYn/pub?start=false&loop=false&delayms=3000)

ToDo

- производная для умножения, производная степенной функции
- добавить информацию про y.grad_fn
- squeeze, unsqueeze, reshape, resize, etc

## Производные в torch.Tensor (отличие от numpy.array)

Отличительной способностью torch является возможность автоматически рассчитывать производные. Каждый тензор, который вы создаете, способен "запоминать" какие действия с ним происходили (перемножение, сложение, возведение в степень), с той целью, чтобы при вызове метода .grad( ) рассчитать значение производной для этого тензора. По умолчанию тензор не ведет запись операций, чтобы изменить это, надо установить значения свойства requires_grad в True. 

Забегая немного вперед, скажем, что во время вычислений с тензором, у которого флаг requires_grad установлен в  True, для него строится вычислительный граф, который даже можно изобразить с помощью дополнительной библиотеки, но об этом позже.

### Производная для одной переменной

Давайте посмотрим как можно пользоваться автоматическим вычислением производной в torch. Для начала импортируем требуемые библиотеки

In [1]:
import torch
import numpy
from PIL import Image
import matplotlib.pyplot as plt

создадим одномерный тензор длинной один

In [2]:
x = torch.Tensor([4])

убедимся, что сейчас тензор не будет "запоминать" происходящих с ним операций

In [3]:
x.requires_grad

False

установим флаг requires_grad в True, для того, чтобы затем можно было рассчитать производную для этого тензора

In [4]:
x.requires_grad = True

произведем вычисления, включающие в себя операции с нашей переменной x

In [5]:
y = x * 4

значение производной хранится в свойстве grad и изначально равно None чтобы рассчитать его, надо вызвать соответствующий метод

In [6]:
x.grad == None

True

метод, который вычисляет производные, называется backward() - что можно перевести как "обратное направление". Называется он так, потому что производные вычисляются из конца в начало и вызывать его надо от тензора, который является результатом функции: в нашем случае от тензора y

In [7]:
y.backward()

теперь в grad хранится значение производной

In [8]:
x.grad

tensor([4.])

### Производная для многомерной функции

В предыдущем примере значение переменной **y** зависело всего от одной переменной **x**, поэтому, вызывая метод **y.backward( )**, производная была вычислена только лишь для **x**. Как вы могли догадаться, если **y** будет зависеть от большего числа переменных, то при вызове **y.bacward( )** производные расчитаются и для всех них. Давайте проверим это.

In [9]:
x_1 = torch.Tensor([3])
x_2 = torch.Tensor([5])
x_3 = torch.Tensor([1])

In [10]:
x_1.requires_grad = True
x_2.requires_grad = True
x_3.requires_grad = True

In [11]:
y = -5*x_1 + 2*x_2 + 7*x_3

In [12]:
y.backward()

In [13]:
x_1.grad, x_2.grad, x_3.grad

(tensor([-5.]), tensor([2.]), tensor([7.]))

<br>
<br>
Что вам напоминает вычисление этой функции?

In [14]:
x = torch.Tensor([3, 5, 1])
x.requires_grad = True

In [15]:
v = torch.Tensor([-5, 2, 7])

In [16]:
y = x.dot(v)

In [17]:
y.backward()

In [18]:
x.grad

tensor([-5.,  2.,  7.])

<br>
<br>
Получается мы можем вычислить производную и для вектора, просто по-отдельности посчитав производные для его компонент. Но вектор - частный случай матрицы, а матрица - частный случай тензора - значит можно вычислять производные и для них. И даже нужно, если мы хотим обучить нейросеть.

In [19]:
v = torch.Tensor([1, 2, -1])

m = torch.Tensor([ 
    [ 1, -1,  0],
    [ 0,  5, -4],
    [-2,  0,  2]
])

p = torch.Tensor([1, 0, 1])

Перемножим вектор v на матрицу m, после этого результат скалярно умножим на вектор p. Но если не изменить размерности векторов, мы не сможем совершить умножение. В torch матрицу можно умножить только на матрицу, вектор на вектор. Вектор отличается от матрицы тем, что у него лишь одна размерность, это можно увидеть, вызвав свойство shape

In [20]:
print(v.shape, '- это форма вектора v, видим, что он одномерный (длинна равна 3)')
print(m.shape, '- это форма матрицы m, видим, что она многомерна (высота и ширины равны 3)')

torch.Size([3]) - это форма вектора v, видим, что он одномерный (длинна равна 3)
torch.Size([3, 3]) - это форма матрицы m, видим, что она многомерна (высота и ширины равны 3)


Чтобы сделать из вектора матрицу с одной строкой, можно вызвать метод **.unsqueeze(dim)** от тензора, dim - какая размность будет являться фиктивной. Unsqueeze можно перевести как "расжать", то есть увеличить размерность. С его помощью можно сделать из одномерного массива двумерный, из двумерного трехмерный или из трехмерного сразу стомерный, добавляя "фиктивные" размерности. Работает это так: пусть у нас есть одномерный массив

```Python
a = [1,2,3]
```

мы можем легко сделать из него двумерный, добавив еще больше скобочек


```Python
a = [[1,2,3]]
```

но ведь можно сделать двумерный массив и другим образом, сделав матрицу с одним столбцом и тремя строками

```Python
a = [[1],[2],[3]]
```

в методе **.unsqueeze(dim)** параметр dim как раз указывает каким образом увеличить размерность (какую размерность сделать фиктивной: первую, вторую или третью и тд).


Метод-антагонист называется как неудивительно **squeeze( )**. Он сжимает размерности, но об этом позже. Вернемся к нашей задаче, мы хотим умножить v на матрицу m. Для этого сделаем из вектора v матрицу, добавив фиктивную размерность и превратим v в матрицу с одной строкой

In [21]:
v = v.unsqueeze(0).requires_grad_(True)
v.shape

torch.Size([1, 3])

In [22]:
p = p.unsqueeze(1).requires_grad_(True)
p.shape

torch.Size([3, 1])

Перемножим v на m и результат на p

In [23]:
y = v.mm(m).mm(p)
y.shape

torch.Size([1, 1])

In [24]:
y.backward()

In [25]:
v.grad

tensor([[ 1., -4.,  0.]])

In [26]:
m.grad

In [27]:
p.grad

tensor([[  3.],
        [  9.],
        [-10.]])

### torch.nn.Module

In [28]:
class MyModel(torch.nn.Module):
    def __init__(self, input_shape):
        super(MyModel, self).__init__()
        
        self.fc1 = torch.nn.Linear(input_shape, 64, bias=True)
        self.fc2 = torch.nn.Linear(64, 32, bias=True)
        self.fc3 = torch.nn.Linear(32, 1, bias=True)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        
        return x

In [29]:
model = MyModel(192)

In [30]:
input_tensor = torch.rand(16, 192)
output = model(input_tensor)

In [31]:
output.shape

torch.Size([16, 1])

In [32]:
output = sum(output)

In [33]:
output.backward()

In [34]:
model.fc1.weight.grad

tensor([[-0.3304, -0.4305, -0.2893,  ..., -0.3323, -0.3678, -0.3558],
        [ 0.3325,  0.4333,  0.2912,  ...,  0.3345,  0.3702,  0.3582],
        [ 0.1751,  0.2281,  0.1533,  ...,  0.1761,  0.1949,  0.1886],
        ...,
        [-0.4805, -0.6262, -0.4208,  ..., -0.4834, -0.5350, -0.5176],
        [ 0.6037,  0.7867,  0.5287,  ...,  0.6073,  0.6721,  0.6502],
        [ 0.0487,  0.0635,  0.0427,  ...,  0.0490,  0.0542,  0.0525]])